### Variables
We have our system state defined as  - 
* SS: (n, nd, (ch1, st1, r1,m, tbm, sbm), (ch2, st2, r2,m, tb2,m, sb2,m),.. (chn, stn, rn,m, tbn,m, sbn,m))
    * n - total number of available buses
    * m - subscript that denotes route m
    * nd - number of buses deployed
    * chn -  charging state of bus n
    * stn - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * rn,m - the route on which the bus is deployed
    * tbn,m - the previous stop the bus travelled to (travel time)
    * sbn,m - the previous stop which was served by the bus (service time)
        * If tbn= sbn, generate travel time for tbn+1; tbn = tbn+1 if accepted
        * Else if tbn> sbn, generate service time for sbn+1; sbn = sbn+1 if accepted

* t - the clock time/current time
* nrm - number of buses required on route m
* btn - time array for monitoring travel time and service time related to each bus


### Pseudo-code
1. Start at t = 0
2. Generate requirements from route, nd (required number of buses):
    1. Check availability of buses & minimum fuel level/battery capacity required for the routes
    2. Choose number of buses, K, to be deployed based on checks
        1. If K<nd, deploy buses with higher fuel/battery level and recharge/refuel the rest
        2. Else, send buses for recharge/refuel
    3. For each bus k, in {1, 2,.. n}, after selecting a specific route m {stops - 1, 2,... M}:
    4. Initialize j = 0 (Depot_Start)
    5. Generate interarrival times for stop j, update time tbk,j
    6. Generate service time at stop j, update time sbk,j
    7. STOP if j = M; Else update j = j + 1, goto 2.b. 
3. Based on the time we update the System State (SS) and the concerned variables.

### Cases
1. We generate demand for the route at time t,
    Sub-task: Generating series of buses to be deployed
    1. Condition 1: number of buses required <= number of charged/fuelled buses
        1. Deploy buses based on descending order of fuel level/charge
        2. Send buses for refuelling/recharging, if they are below a certain threshold
    2. Condition 2: number of buses required > number of charged/fuelled buses
        1. Send buses for refuelling/recharging
        2. As soon as the bus finishes recharging/refueling, buses get sent to the route
2. Routing of buses:
    1. Route 0: based on type of the bus and refuelling need. Will be a refuelling/ recharging route for the bus
    2. Route 1 to m: Buses will go through the stops till they reach Depot_Stop


## Variables
#### Buses
* bus: ($ch_n, st_n, r_{n,m}, tb_{n,m}, sb_{n,m}$)
    * $ch_n$ - charging state of bus n
    * $st_n$ - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * $r_{n,m}$ - the route on which the bus is deployed
    * $tb_{n,m}$ - the previous stop the bus travelled to (travel time)
    * $sb_{n,m}$ - the previous stop which was served by the bus (service time)
        * If $tb_n$= $sb_n$, generate travel time for $tb_{n+1}$; $tb_n$ = $tb_{n+1}$ if accepted
        * Else if $tb_n$> $sb_n$, generate service time for $sb_{n+1}$; $sb_n$ = $sb_{n+1}$ if accepted
    * Arrival time - route generated arrays
    * Service Time - route generated arrays
    * 

___
### System State New
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill)
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill, route_1_buses, route_2_buses,....)

#### Monitoring System State 

| Time | System State | Bus | Charge | Route | State | Event | Process Time | Demand-Current | Demand-Actual | Demand-Charge |
| ---- | ------------ | --- | ------ | ----- | ----- | ----- | ------------ | -------------- | ------------- | ------------- |
| 0    | (1,1,1,0,0)  | 1   | 50     | '1'   | 1     | 0     | 0            | 0              | 0             | 15            |
| 10.3 | (1,0,1,0,0)  | 1   | 48     | '1'   | 1     | 1     | 10.3         | -              | -             | -             |
| 12.3 | (1,0,1,0,0)  | 1   | 47.5   | '1'   | 1     | 0     | 2.0          | -              | -             | -             |

___

#### Monitoring Bus Deployments

| Time | Demand-Current | Demand-Actual | Demand-Charge | Bus | Charge | Route | Event Array  | Time Array   |
| ---- | -------------- | ------------- | ------------- | --- | ------ | ----- | ------------ | ------------ |
| 0    | 0              | 0             | 15            | 1   | 50     | '1'   | [1, 0, 1]    | [12, 35, 41] |
| 40   | 41             | 40            | 20            | 1   | 35     | '2'   | [1, 0, 1]    | [48, 50, 55] |



In [3]:
import pandas as pd
import numpy as np
import numpy.random as random
from route_functions import *
import matplotlib.pyplot as plt

In [4]:
routes = pd.read_excel('Model_Parameters.xlsx', 'Routes')
SS_cols = ['Time', 'System_State', 'Bus', 'Charge', 'Route', 
           'State', 'Event', 'Process_Time', 'Demand_Current', 
           'Demand_Actual', 'Demand_Charge']
BD_cols = ['Time', 'Demand_Current', 'Demand_Actual', 'Demand_Charge', 
           'Bus', 'Charge', 'Route', 'Event_Array', 'Time_Array']

In [13]:
# customizable parameters
spare_buses = 2
average_bus_speed = 10
running_consumption = 0.5             # fuel(lts.)/kWh per minute
service_consumption = 0.5
refuel_stations = 2
refuel_consumption = -10

# tank level/charge of buses
level_mean = 50       # normal distribution.. can be changed
level_std = 2.5
refuel = 'refill'     # 'recharge'

# setting up initial values for simulation
n_buses = 6                                # number of buses
n_routes = 2                               # number of routes
buses = [bus(level_mean, level_std) for i in range(n_buses)]    # fleet of buses
t = 0                                      # start time of simulation
T = 720

# updating demands
demand_at, demand_r, demand_c = gen_demands(n_routes, T)
demand_ct = demand_at.copy()         # demand current time and demand actual time
dct_flag = 0                  # demand at current time

ss_table = pd.DataFrame(columns=SS_cols)
bd_table = pd.DataFrame(columns=BD_cols)

'''# -----
#### creating fake data & demands
t = 0                                      # start time of simulation
T = 60
buses = [bus(5, 5) for i in range(1)]
demand_at = [0, 40, np.inf]
demand_r = ['1', '2', None]
demand_c = [15, 20, np.inf]
demand_ct = demand_at.copy()         # demand current time and demand actual time
dct_flag = 0                  # demand at current time'''


# -----
# Initial SS update
time_check = np.inf
ss_table = SS_update(ss_table, t, buses, dct_flag, np.nan, np.nan)

# Simulate! Simulate! Simulate!        
while (t<T)or(time_check!=np.inf):
    print('\n---\nNew Loop')
    
    time_check = min(next_bus_e(buses)[0], demand_ct[0])
    print('Time check -', time_check)
    
    # -----
    # Priority One: Updating demands
    if ((demand_ct[0]!=np.inf) and
        (demand_ct[0]==time_check)):
        new_demand = [1 for i in range(len(demand_ct)) if demand_ct[i]==time_check]
        dct_flag = sum(new_demand)
        
    print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
    print('\tdct_flag -', dct_flag)
    
    
    #----------
    # Main Simulation
    bus_chk = available_bus(buses, demand_c[0])
    print('Bus Check -', bus_chk)
    
    
    #-----
    # Case 1: Sending low-fuel buses to refuel
    refuel_index = unavailable_bus(buses, min(demand_c))
    if (#(t<T) and
        (refuel_index!=-1) and
        (buses_status(buses)[1]<refuel_stations)): 
        # or((dct_flag>0)and(refuel_index!=np.inf)):- might create complications
        dem_ct, dem_at, dem_c = np.nan, np.nan, np.nan
        print('Sending Bus for Refuel')
        buses[refuel_index].assign_route(routes, refuel, t)
        bus_e = 0
        t_updt = t
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, refuel_index)
        bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, refuel_index)
        
        t = t_updt
        refuel_index = -1
    
    
    #-----
    # Case 2: Checking bus availability and deploying buses
    elif ((t<T) and
          (demand_ct[0]==time_check) and
          (demand_ct[0]!=np.inf) and
          (bus_chk!=-1)):
        
        print('Deploying Bus')
        dem_ct, dem_at = demand_ct.pop(0), demand_at.pop(0)
        dem_c, dem_r = demand_c.pop(0), demand_r.pop(0)
        index = available_bus(buses, dem_c)
        buses[index].assign_route(routes, dem_r, t)
        t_updt = dem_ct
        bus_e = 0
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index, dem_ct, dem_at, dem_c)
        bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, index)
        print(buses[index].event_arr, buses[index].time_arr)
        dct_flag -= 1
        t = t_updt
        
        dem_ct, dem_at, dem_c, dem_r = np.nan, np.nan, np.nan, np.nan
        t_updt = np.nan
        index = np.nan
        
    
    #-----
    # Case 3: Checking next bus event and updating SS
    elif ((t<T) and
          (next_bus_e(buses)[0]==time_check) and 
          (next_bus_e(buses)[0]!=np.inf) and 
          ((buses_status(buses)[0]>0) or (buses_status(buses)[1]>0))):
        
        print('Updating next bus event')
        index = next_bus_e(buses)[2]
        t_updt = buses[index].time_arr.pop(0)
        bus_e = buses[index].event_arr.pop(0)
        
        if (bus_e==0)and(buses[index].state==1):
            mul_c = service_consumption
        elif (bus_e==1):
            mul_c = running_consumption
        elif (bus_e==0)and(buses[index].state==0):            # different for 'refill' and 'recharge' 
            mul_c = refuel_consumption                        # difference from 90 for refill
        buses[index].charge = buses[index].charge - (t_updt - t)*mul_c
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index)
        t = t_updt
        
        # if its the last event for the bus, update bus parameters to standstill
        if buses[index].next_e()==np.inf:
            buses[index].state = -1
            buses[index].route = None
            buses[index].time_arr = list()
            buses[index].event_arr = list()
        index = np.nan
        t_updt = np.nan
        bus_e = np.nan
        
        
    #-----
    # Case 4: Jumping to next demand 
    elif ((t<T) and
          (demand_ct[0]==time_check) and 
          (demand_ct[0]!=np.inf) and 
          (bus_chk==-1)):
        t = demand_ct[dct_flag]
    
    
    #-----
    # Case 5: All demands are completed within the timeframe 
    elif ((t<T) and
          (time_check==np.inf)):
        
        print('Jumping to EOD')
        t_updt = T
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt)
        t = t_updt    
    
    
    #-----
    # Case 6: Demands still exist beyond timeframe and need to be met
    elif ((t>T) and
          (demand_ct[0]==time_check) and
          (demand_ct[0]!=np.inf) and
          (bus_chk!=-1)):
        
        print('Deploying Bus beyond timeframe')
        dem_ct, dem_at = demand_ct.pop(0), demand_at.pop(0)
        dem_c, dem_r = demand_c.pop(0), demand_r.pop(0)
        index = available_bus(buses, dem_c)
        buses[index].assign_route(routes, dem_r, t)
        t_updt = dem_ct
        bus_e = 0
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index, dem_ct, dem_at, dem_c)
        bd_table = BD_update(bd_table, t_updt, dem_ct, dem_at, dem_c, buses, index)
        dct_flag -= 1
        t = t_updt
        
        dem_ct, dem_at, dem_c, dem_r = np.nan, np.nan, np.nan, np.nan
        t_updt = np.nan
        index = np.nan
        

    #-----
    # Case 7: Going beyond timeframe, checking deployed buses and updating SS
    elif ((t>T) and
          (next_bus_e(buses)[0]==time_check) and
          (next_bus_e(buses)[0]!=np.inf)):
        print('Updating next bus event beyond timeframe')
        index = next_bus_e(buses)[2]
        t_updt = buses[index].time_arr.pop(0)
        bus_e = buses[index].event_arr.pop(0)
        
        if (bus_e==0)and(buses[index].state==1):
            mul_c = service_consumption
        elif (bus_e==1):
            mul_c = running_consumption
        elif (bus_e==0)and(buses[index].state==0):            # different for 'refill' and 'recharge' 
            mul_c = refuel_consumption                        # difference from 90 for refill
        buses[index].charge = buses[index].charge - (t_updt - t)*mul_c
        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt, index)
        t = t_updt
        
        # if its the last event for the bus, update bus parameters to standstill
        if buses[index].next_e()==np.inf:
            buses[index].state = -1
            buses[index].route = None
            buses[index].time_arr = list()
            buses[index].event_arr = list()
        index = np.nan
        t_updt = np.nan
        bus_e = np.nan
    
    #-----
    # Case 8: All demands are completed within the timeframe 
    elif ((t>T) and
          (time_check==np.inf)):
        
        print('Jumping to EOD')
        t_updt = np.inf        
        ss_table = SS_update(ss_table, t, buses, dct_flag, bus_e, t_updt)
        t = t_updt    
    
    
    
    print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
    print('\tdct_flag -', dct_flag)
    
    # updating the current demand - if demand exists it'll automatically be updated
    for k in range(dct_flag):
        demand_ct[k] = t
        
    print('\tDemands -', demand_ct, demand_at, demand_r, demand_c)
    print('\tdct_flag -', dct_flag)
    print('t -', t)


---
New Loop
Time check - 0.0
	Demands - [0.0, 0.0, 20.0, 30.0, 40.0, 60.0, 60.0, 72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0, 432.0, 444.0, 450.0, 456.0, 468.0, 480.0, 480.0, 492.0, 495.0, 504.0, 510.0, 516.0, 525.0, 528.0, 540.0, 540.0, 552.0, 555.0, 564.0, 570.0, 576.0, 585.0, 588.0, 600.0, 600.0, 612.0, 620.0, 624.0, 636.0, 640.0, 648.0, 660.0, 660.0, 672.0, 684.0, 690.0, 696.0, 708.0, inf] [0.0, 0.0, 20.0, 30.0, 40.0, 60.0, 60.0, 72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.

C:\Users\krishrao\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


	Demands - [20.0, 30.0, 40.0, 60.0, 60.0, 72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0, 432.0, 444.0, 450.0, 456.0, 468.0, 480.0, 480.0, 492.0, 495.0, 504.0, 510.0, 516.0, 525.0, 528.0, 540.0, 540.0, 552.0, 555.0, 564.0, 570.0, 576.0, 585.0, 588.0, 600.0, 600.0, 612.0, 620.0, 624.0, 636.0, 640.0, 648.0, 660.0, 660.0, 672.0, 684.0, 690.0, 696.0, 708.0, inf] [20.0, 30.0, 40.0, 60.0, 60.0, 72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0, 432.0, 444.0, 450.0, 456.0, 468.0, 480.0, 480.0,

[1, 0, 1, 0, 1, 0, 1] [41.4371, 41.8595, 59.7329, 61.1869, 70.5061, 70.8728, 73.1774]
	Demands - [60.0, 60.0, 72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0, 432.0, 444.0, 450.0, 456.0, 468.0, 480.0, 480.0, 492.0, 495.0, 504.0, 510.0, 516.0, 525.0, 528.0, 540.0, 540.0, 552.0, 555.0, 564.0, 570.0, 576.0, 585.0, 588.0, 600.0, 600.0, 612.0, 620.0, 624.0, 636.0, 640.0, 648.0, 660.0, 660.0, 672.0, 684.0, 690.0, 696.0, 708.0, inf] [60.0, 60.0, 72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0

[1.0, 0.0, 1.0, 0.0, 1.0] [64.9057, 64.9177, 87.1718, 87.596, 93.8619]
	Demands - [72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0, 432.0, 444.0, 450.0, 456.0, 468.0, 480.0, 480.0, 492.0, 495.0, 504.0, 510.0, 516.0, 525.0, 528.0, 540.0, 540.0, 552.0, 555.0, 564.0, 570.0, 576.0, 585.0, 588.0, 600.0, 600.0, 612.0, 620.0, 624.0, 636.0, 640.0, 648.0, 660.0, 660.0, 672.0, 684.0, 690.0, 696.0, 708.0, inf] [72.0, 84.0, 90.0, 96.0, 108.0, 120.0, 120.0, 132.0, 140.0, 144.0, 156.0, 160.0, 168.0, 180.0, 180.0, 192.0, 200.0, 204.0, 216.0, 220.0, 228.0, 240.0, 240.0, 255.0, 260.0, 270.0, 280.0, 285.0, 300.0, 300.0, 312.0, 324.0, 330.0, 336.0, 348.0, 360.0, 360.0, 372.0, 384.0, 390.0, 396.0, 408.0, 420.0, 420.0, 432.0, 444.0, 450.0, 456.0, 468.0, 48

...

	Demands - [684.0, 690.0, 696.0, 708.0, inf] [684.0, 690.0, 696.0, 708.0, inf] ['1', '2', '1', '1', None] [25.0, 20.0, 25.0, 25.0, inf]
	dct_flag - 0
	Demands - [684.0, 690.0, 696.0, 708.0, inf] [684.0, 690.0, 696.0, 708.0, inf] ['1', '2', '1', '1', None] [25.0, 20.0, 25.0, 25.0, inf]
	dct_flag - 0
t - 674.9649

---
New Loop
Time check - 675.256
	Demands - [684.0, 690.0, 696.0, 708.0, inf] [684.0, 690.0, 696.0, 708.0, inf] ['1', '2', '1', '1', None] [25.0, 20.0, 25.0, 25.0, inf]
	dct_flag - 0
Bus Check - -1
Updating next bus event
	Demands - [684.0, 690.0, 696.0, 708.0, inf] [684.0, 690.0, 696.0, 708.0, inf] ['1', '2', '1', '1', None] [25.0, 20.0, 25.0, 25.0, inf]
	dct_flag - 0
	Demands - [684.0, 690.0, 696.0, 708.0, inf] [684.0, 690.0, 696.0, 708.0, inf] ['1', '2', '1', '1', None] [25.0, 20.0, 25.0, 25.0, inf]
	dct_flag - 0
t - 675.256

---
New Loop
Time check - 675.3391
	Demands - [684.0, 690.0, 696.0, 708.0, inf] [684.0, 690.0, 696.0, 708.0, inf] ['1', '2', '1', '1', None] [25.0, 20

New Loop
Time check - 685.7783
	Demands - [690.0, 696.0, 708.0, inf] [690.0, 696.0, 708.0, inf] ['2', '1', '1', None] [20.0, 25.0, 25.0, inf]
	dct_flag - 0
Bus Check - -1
Updating next bus event
	Demands - [690.0, 696.0, 708.0, inf] [690.0, 696.0, 708.0, inf] ['2', '1', '1', None] [20.0, 25.0, 25.0, inf]
	dct_flag - 0
	Demands - [690.0, 696.0, 708.0, inf] [690.0, 696.0, 708.0, inf] ['2', '1', '1', None] [20.0, 25.0, 25.0, inf]
	dct_flag - 0
t - 685.7783

---
New Loop
Time check - 686.2306
	Demands - [690.0, 696.0, 708.0, inf] [690.0, 696.0, 708.0, inf] ['2', '1', '1', None] [20.0, 25.0, 25.0, inf]
	dct_flag - 0
Bus Check - -1
Updating next bus event
	Demands - [690.0, 696.0, 708.0, inf] [690.0, 696.0, 708.0, inf] ['2', '1', '1', None] [20.0, 25.0, 25.0, inf]
	dct_flag - 0
	Demands - [690.0, 696.0, 708.0, inf] [690.0, 696.0, 708.0, inf] ['2', '1', '1', None] [20.0, 25.0, 25.0, inf]
	dct_flag - 0
t - 686.2306

---
New Loop
Time check - 686.2417
	Demands - [690.0, 696.0, 708.0, inf] [690.

	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
t - 696.6584

---
New Loop
Time check - 696.9651
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
Bus Check - -1
Updating next bus event
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
t - 696.9651

---
New Loop
Time check - 697.6767
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
Bus Check - -1
Updating next bus event
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
t - 697.6767

---
New Loop
Time check - 697.8021
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 0
Bus Check - -1
Updating next bus event
	Demands - [708.0, inf] [708.0, inf] ['1', None] [25.0, inf]


C:\Users\krishrao\AppData\Local\Temp/ipykernel_10364/1531347533.py:212: RuntimeWarning: invalid value encountered in double_scalars
  buses[index].charge = buses[index].charge - (t_updt - t)*mul_c


 [inf, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
	Demands - [712.5425, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
t - 712.5425

---
New Loop
Time check - 712.5425
	Demands - [712.5425, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
Bus Check - -1
	Demands - [712.5425, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
	Demands - [inf, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
t - inf

---
New Loop
Time check - 712.974
	Demands - [inf, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
Bus Check - -1
Updating next bus event beyond timeframe
	Demands - [inf, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
	Demands - [712.974, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
t - 712.974

---
New Loop
Time check - 712.974
	Demands - [712.974, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
Bus Check - -1
	Demands - [712.974, inf] [708.0, inf] ['1', None] [25.0, inf]
	dct_flag - 1
	Demands - [inf, inf] [708.0

In [20]:
ss_table[ss_table['Bus']==3][20:50]

,Time,System_State,Bus,Charge,Route,State,Event,Process_Time,Demand_Current,Demand_Actual,Demand_Charge
114,152.9384,"[6, 0, 4, 0, 2]",3,38.312279,2,1,0.0,0.2375,NaN,NaN,NaN
119,159.5431,"[6, 0, 4, 0, 2]",3,37.517179,2,1,1.0,1.5902,NaN,NaN,NaN
128,168.0000,"[6, 1, 4, 0, 2]",3,37.517179,1,1,0,1.5577,168.000,168.0,25.0
130,171.6190,"[6, 0, 4, 0, 2]",3,37.357279,1,1,1,0.3198,NaN,NaN,NaN
131,171.9733,"[6, 0, 4, 0, 2]",3,37.180129,1,1,0,0.3543,NaN,NaN,NaN
146,186.6503,"[6, 0, 5, 0, 1]",3,36.158129,1,1,1,2.0440,NaN,NaN,NaN
147,186.7976,"[6, 0, 5, 0, 1]",3,36.084479,1,1,0,0.1473,NaN,NaN,NaN
152,194.5015,"[6, 0, 5, 0, 1]",3,34.301079,1,1,1,3.5668,NaN,NaN,NaN
154,197.3114,"[6, 0, 4, 0, 2]",3,33.054779,1,1,0,2.4926,NaN,NaN,NaN
169,212.7434,"[6, 0, 6, 0, 0]",3,31.070429,1,1,1,3.9687,NaN,NaN,NaN


In [15]:
bd_table

,Time,Demand_Current,Demand_Actual,Demand_Charge,Bus,Charge,Route,Event_Array,Time_Array,State
0,0.0000,0.0000,0.0,25.0,6,52.093059,1,"[1, 0, 1, 0, 1, 0, 1]","[3.1057, 3.8686, 16.4262, 16.5257, 27.8916, 28...",1.0
1,0.0000,0.0000,0.0,20.0,2,49.784677,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[4.2052, 6.8109, 19.1226, 20.2709, 27.5781]",1.0
2,20.0000,20.0000,20.0,25.0,3,49.705629,1,"[1, 0, 1, 0, 1, 0, 1]","[26.8378, 27.0363, 42.5774, 42.9857, 54.157, 5...",1.0
3,30.0000,30.0000,30.0,20.0,5,49.535190,2,"[1.0, 0.0, 1.0, 0.0, 1.0]","[34.2033, 34.5069, 56.4292, 56.5418, 66.442]",1.0
4,40.0000,40.0000,40.0,25.0,1,48.433478,1,"[1, 0, 1, 0, 1, 0, 1]","[41.4371, 41.8595, 59.7329, 61.1869, 70.5061, ...",1.0
...,...,...,...,...,...,...,...,...,...,...
165,695.3399,NaN,NaN,NaN,3,-2091.530021,refill,"[1.0, 0.0, 1.0]","[695.6588, 696.6584, 697.8021]",0.0
166,697.8021,NaN,NaN,NaN,3,-2081.756171,refill,"[1.0, 0.0, 1.0]","[698.5063, 699.1748, 699.6956]",0.0
167,699.6956,NaN,NaN,NaN,3,-2077.292121,refill,"[1.0, 0.0, 1.0]","[700.7293, 702.3291, 703.79]",0.0
168,703.7900,NaN,NaN,NaN,3,-2070.616821,refill,"[1.0, 0.0, 1.0]","[703.827, 704.7918, 705.1939]",0.0


In [12]:
bd_table['Time_Array'][60]

array([474.4546, 474.7034, 492.8495, 496.0882, 502.5579])

___
## Testing Area

In [7]:
demand_at, demand_r, demand_c = gen_demands(n_routes, T)
demand_at

[0.0, 0.0, 20.0, 30.0, 40.0, inf]

In [8]:
demand_at.pop(0)
demand_at

[0.0, 20.0, 30.0, 40.0, inf]

In [9]:
# testing for demand creation and updation

#demand = 0
arr = [0, 1, 0, 1, 2, 3]
test = 5
new_demand = [1 for i in range(len(arr)) if arr[i]==test]
demand = demand + sum(new_demand)
demand

NameError: name 'demand' is not defined

In [ ]:
routes[routes.columns[routes.columns.str.match('route_1')]]

In [ ]:
print(route_time(routes, 'recharge', 0))
print(route_time(routes, 'refill', 0))
print(route_time(routes, '2', 0))

In [ ]:
n_buses = 4
buses = [bus(50, 5) for i in range(n_buses)]
t = 10
buses[0].assign_route(routes, 'refill', t)
buses[1].assign_route(routes, '1', t)
buses[2].assign_route(routes, '1', t)
print(buses[0].time_arr)
print(buses[0].event_arr)
buses_status(buses)

In [ ]:
print(buses[0].next_t())
print(buses[0].next_e())
print(buses[0].time_arr.pop(0))
print(buses[0].event_arr.pop(0))
print(buses[0].next_t())
print(buses[0].next_e())

In [ ]:
#### ASSUMPTION: buses are prioritized based on their index

def next_bus_e(buses):
    min_t = np.inf
    index = None
    for i in range(len(buses)):
        print(buses[i].next_t())
        if (buses[i].next_t()<min_t):
            min_t = buses[i].next_t()
            index = i
    return min_t, buses[i].next_e(), index

# testing 
buses = [bus(50, 5) for i in range(3)]
buses[0].assign_route(routes, '1', 0)
buses[1].assign_route(routes, '1', 0)
buses[2].assign_route(routes, '1', 0)
print(buses[0].time_arr)
print(buses[0].event_arr)
print(buses[1].time_arr)
print(buses[1].event_arr)
print(buses[2].time_arr)
print(buses[2].event_arr)
next_bus_e(buses)

def available_bus(buses, dem_charge):
    print(buses)
    b_charges = [buses[i].charge for i in range(len(buses)) if buses[i].state==-1]
    b_index = [i for i in range(len(buses)) if buses[i].state==-1]
    index = np.nan
    if len(b_charges)>0:
        if (max(b_charges)>dem_charge):
            for i in range(len(b_charges)):
                if max(b_charges)==b_charges[i]:
                    index = b_index[i]
    return index

check = bus(20, 4)
buses = [bus(25, 5) for i in range(1)]
buses[0].state = 1
available_bus(buses, 20)

#### Testing for route demands

In [ ]:
# route demand functions
array_t = np.array(range(0, 12*60+1, 60))

a, b, c, d = 2, 3, 0, 75
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='r', label='bus_1', marker='o')

a, b, c, d = 2, 3, -180, 120
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='b', label='bus_2', marker='o')

a, b, c, d = 1, 3, 0, 45
plt.plot(array_t, np.ceil(a*np.sin((array_t+c)/d) + b), 
         color='g', label='bus_3', marker='o')

plt.xlabel('Time')
plt.ylabel('Buses')
plt.legend()
plt.show()